In [1]:
from tkinter import *
import cv2
import numpy as np
import os
from setuptools import setup
import pickle
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.models import load_model

In [4]:
face_cascade = cv2.CascadeClassifier('./HaarCascade/haarcascade_frontalface_default.xml')

def faceRecord():
    
    cap = cv2.VideoCapture(1)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('./PersonalFace/out.avi', fourcc, 20.0, (640, 480))

    while True:
        ret, frame = cap.read()
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        out.write(frame)
        cv2.imshow("image", frame)

        if cv2.waitKey(1) == 13:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    cap = cv2.VideoCapture('./PersonalFace/sample.mp4')
    

    currentFrame = 0
    name = input("Enter Your Name: ")
    dirs = './PersonalFace/images/'+str(name)
    # print(dir)
    if os.path.exists(dirs):
        pass
    else:
        os.makedirs(dirs)
    if name:
        while True:
            ret, frame = cap.read()

            dirs = './PersonalFace/images/'+str(name)+'/'+str(currentFrame)+'.jpg'
            cv2.imwrite(dirs, frame)

            currentFrame += 1
            if cv2.waitKey(1) == 13:
                break
    
    cap.release()
    cv2.destroyAllWindows()

def trainFace():

    
    
    BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
    BASE_DIR = BASE_DIR+'/PersonalFace'
    print(BASE_DIR)
    img_dir = os.path.join(BASE_DIR, "images")
    print(img_dir)

    ##Loading images and cropping the image
    i = 0
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                img = cv2.imread(path)
                img = cv2.resize(img, (280, 280))
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                for (x, y, w, h) in faces:
                        img_name = str(root)+"/"+str(i)+".jpg"
                        print(img_name)
                        roi_color = img[y:y+h+20, x:x+w+20]
                        cv2.imwrite(img_name, roi_color)
                        #cv2.imshow("Frame",roi_color)
                        i += 1
                cv2.waitKey(0)
    
    current_ids = 0
    label_ids = {}
    x_train = []
    y_labels = []
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                label = os.path.basename(root).replace(' ', "-")
                if label in label_ids:
                    pass
                else:
                    label_ids[label] = current_ids
                    current_ids += 1
                id_ = label_ids[label]
                #print(label_ids)
                pil_image = Image.open(path).convert("L")
                image_array = np.array(pil_image, 'uint8')
                #print(image_array)
                faces = face_cascade.detectMultiScale(image_array, 1.5, 5)

                for(x, y, w, h) in faces:
                    roi = image_array[y:y+h, x:x+w]
                    x_train.append(roi)
                    y_labels.append(id_)

    # print(x_train)
    # print(y_labels)

    #############Saving the model and label using pickle ##############

    recognizer = cv2.face.LBPHFaceRecognizer_create()

    with open("./PersonalFace/trained model/label.pickle", "wb") as f:
        pickle.dump(label_ids, f)

    recognizer.train(x_train, np.array(y_labels))
    recognizer.save("./PersonalFace/trained model/trainner.yml")

def personalFace():
    eye_cascade = cv2.CascadeClassifier('./HaarCascade/haarcascade_eye.xml')
    smile_cascade = cv2.CascadeClassifier('./HaarCascade/haarcascade_smile.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("./PersonalFace/trained model/trainner.yml")

    labels = {}
    with open('./PersonalFace/trained model/label.pickle', 'rb') as f:
        og_labels = pickle.load(f)
        labels = {v:k for k, v in og_labels.items()}

    cap = cv2.VideoCapture(1)
    i = 0
    stroke = 2
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            #print(x, y, w, h)
            img_item = str(i)+".png"
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            id_, conf = recognizer.predict(roi_gray)
            #print(labels[id_])
            cv2.putText(frame, labels[id_], (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), stroke)

            color= (0, 0, 255) #BGR
            stroke = 2
            width = x + w
            height = y + h
            cv2.rectangle(frame, (x - 10, y), (width + 10, height + 20), color, stroke)
            eyes = eye_cascade.detectMultiScale(roi_gray)
            #for (ex, ey, ew, eh) in eyes:
            #   cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0))
            # smile = smile_cascade.detectMultiScale(roi_gray)
            # for (sx, sy, sw, sh) in smile:
            #     cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), (255, 0, 0))

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()

def detectEmotion():
    num_classes = 6
    img_rows, img_cols = 48, 48
    batch_size = 16

    train_data_dir = './images/train'
    validation_data_dir = './images/validation'

    validation_datagen = ImageDataGenerator(rescale=1./255)

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode='grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False    
    )

    class_labels = validation_generator.class_indices
    class_labels = {v: k for k, v in class_labels.items()}
    classes = list(class_labels.values())
    print(class_labels)
    
    classifier = load_model('./TrainedModel/4(67)/Emotion_detector.h5')    
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('./PersonalFace/trained model/trainner.yml')

    labels = {}
    with open('./PersonalFace/trained model/label.pickle', 'rb') as f:
        og_label = pickle.load(f)
        labels = {v: k for k, v in og_label.items()}
    # print(labels)

    def face_detector(img, gray):
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if faces == ():
            return (0, 0, 0, 0), np.zeros((48, 48), np.uint8), img

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x-30, y-30), (x+w+20, y+h+20), (255, 0, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]

        try:
            roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        except:
            return (x, y, w, h), np.zeros((48, 48), np.uint8), img

        return (x, y, w, h), roi_gray, img

    cap = cv2.VideoCapture(1)

    while True:

        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rect, face, image = face_detector(frame, gray)
        if np.sum([face]) != 0.0:
            roi = face.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            preds = classifier.predict(roi)[0]

            roi_gray = gray[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
            id_, conf = recognizer.predict(roi_gray)

            label = labels[id_]+" is "+class_labels[preds.argmax()]
            label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
            cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        else:
            cv2.putText(image, "No Face found", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)

        cv2.imshow('Emotion Detection', image)
        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()


root = Tk()

root.title("Personal Face Recognition and Emotion-Recognition")
root.geometry('500x570')
tf = Frame(root, width = 480, height = 50, bd = 2, relief = "ridge")
tf.place(x = 10, y = 10)
title = Label(tf,font = ("Helvetica",30),fg = "blue",text = "       Emotion Detection      ")
title.pack() 

button1 = Button(root, command = faceRecord, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Open webcam for personal face detection", bd = 5)
button1.place(x = 20, y = 80)

button2 = Button(root, command = trainFace, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Train your face", bd = 5)
button2.place(x = 20, y = 160)

button3 = Button(root, command = personalFace, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Your Personal Face", bd = 5)
button3.place(x = 20, y = 240)

button4 = Button(root, command = detectEmotion, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Emotion with your face", bd = 5)
button4.place(x = 20, y = 320)

root.mainloop()

Found 216 images belonging to 6 classes.
{0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\agnelselvan\appdata\local\programs\python\python37\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-39dd62a1b845>", line 190, in detectEmotion
    recognizer.read('./PersonalFace/trained model/trainner.yml')
cv2.error: OpenCV(4.0.0) C:\projects\opencv-python\opencv_contrib\modules\face\src\facerec.cpp:61: error: (-2:Unspecified error) File can't be opened for reading! in function 'cv::face::FaceRecognizer::read'



In [4]:
face_cascade = cv2.CascadeClassifier('./HaarCascade/haarcascade_frontalface_default.xml')

def faceRecord():
    
    cap = cv2.VideoCapture(1)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('./PersonalFace/out.avi', fourcc, 20.0, (640, 480))

    while True:
        ret, frame = cap.read()
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        out.write(frame)
        cv2.imshow("image", frame)

        if cv2.waitKey(1) == 13:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    cap = cv2.VideoCapture('./PersonalFace/sample.mp4')
    

    currentFrame = 0
    name = input("Enter Your Name: ")
    dirs = './PersonalFace/images/'+str(name)
    # print(dir)
    if os.path.exists(dirs):
        pass
    else:
        os.makedirs(dirs)
    if name:
        while True:
            ret, frame = cap.read()

            dirs = './PersonalFace/images/'+str(name)+'/'+str(currentFrame)+'.jpg'
            
            image = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE) 
            cv2.imwrite(dirs, image)

            currentFrame += 1
            if cv2.waitKey(1) == 13:
                break
    
    cap.release()
    cv2.destroyAllWindows()

def trainFace():
    BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
    BASE_DIR = BASE_DIR+'/PersonalFace'
    print(BASE_DIR)
    img_dir = os.path.join(BASE_DIR, "images")
    print(img_dir)

    ##Loading images and cropping the image
    i = 0
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                img = cv2.imread(path)
                img = cv2.resize(img, (280, 280))
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                for (x, y, w, h) in faces:
                        img_name = str(root)+"/"+str(i)+".jpg"
                        print(img_name)
                        roi_color = img[y:y+h+20, x:x+w+20]
                        cv2.imwrite(img_name, roi_color)
                        #cv2.imshow("Frame",roi_color)
                        i += 1
                cv2.waitKey(0)
    
    current_ids = 0
    label_ids = {}
    x_train = []
    y_labels = []
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                label = os.path.basename(root).replace(' ', "-")
                if label in label_ids:
                    pass
                else:
                    label_ids[label] = current_ids
                    current_ids += 1
                id_ = label_ids[label]
                #print(label_ids)
                pil_image = Image.open(path).convert("L")
                image_array = np.array(pil_image, 'uint8')
                #print(image_array)
                faces = face_cascade.detectMultiScale(image_array, 1.5, 5)

                for(x, y, w, h) in faces:
                    roi = image_array[y:y+h, x:x+w]
                    x_train.append(roi)
                    y_labels.append(id_)

    # print(x_train)
    # print(y_labels)

    #############Saving the model and label using pickle ##############

    recognizer = cv2.face.LBPHFaceRecognizer_create()

    with open("./PersonalFace/trainedmodel/label.pickle", "wb") as f:
        pickle.dump(label_ids, f)

    recognizer.train(x_train, np.array(y_labels))
    recognizer.save("./PersonalFace/trainedmodel/trainner.yml")

def personalFace():
    eye_cascade = cv2.CascadeClassifier('./HaarCascade/haarcascade_eye.xml')
    smile_cascade = cv2.CascadeClassifier('./HaarCascade/haarcascade_smile.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("./PersonalFace/trainedmodel/trainner.yml")

    labels = {}
    with open('./PersonalFace/trainedmodel/label.pickle', 'rb') as f:
        og_labels = pickle.load(f)
        labels = {v:k for k, v in og_labels.items()}

    cap = cv2.VideoCapture(1)
    i = 0
    stroke = 2
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            #print(x, y, w, h)
            img_item = str(i)+".png"
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            id_, conf = recognizer.predict(roi_gray)
            #print(labels[id_])
            cv2.putText(frame, labels[id_], (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), stroke)

            color= (0, 0, 255) #BGR
            stroke = 2
            width = x + w
            height = y + h
            cv2.rectangle(frame, (x - 10, y), (width + 10, height + 20), color, stroke)
            eyes = eye_cascade.detectMultiScale(roi_gray)
            #for (ex, ey, ew, eh) in eyes:
            #   cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0))
            # smile = smile_cascade.detectMultiScale(roi_gray)
            # for (sx, sy, sw, sh) in smile:
            #     cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), (255, 0, 0))

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()

def detectEmotion():
    num_classes = 6
    img_rows, img_cols = 48, 48
    batch_size = 16

    train_data_dir = './images/train'
    validation_data_dir = './images/validation'

    validation_datagen = ImageDataGenerator(rescale=1./255)

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        color_mode='grayscale',
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False    
    )

    class_labels = validation_generator.class_indices
    class_labels = {v: k for k, v in class_labels.items()}
    classes = list(class_labels.values())
    print(class_labels)
    
    classifier = load_model('./TrainedModel/4(67)/Emotion_detector.h5')    
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('./PersonalFace/trainedmodel/trainner.yml')

    labels = {}
    with open('./PersonalFace/trainedmodel/label.pickle', 'rb') as f:
        og_label = pickle.load(f)
        labels = {v: k for k, v in og_label.items()}
    # print(labels)

    def face_detector(img, gray):
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if faces == ():
            return (0, 0, 0, 0), np.zeros((48, 48), np.uint8), img

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x-30, y-30), (x+w+20, y+h+20), (255, 0, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]

        try:
            roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        except:
            return (x, y, w, h), np.zeros((48, 48), np.uint8), img

        return (x, y, w, h), roi_gray, img

    cap = cv2.VideoCapture(1)

    while True:

        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rect, face, image = face_detector(frame, gray)
        if np.sum([face]) != 0.0:
            roi = face.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            preds = classifier.predict(roi)[0]

            roi_gray = gray[rect[1]:rect[1]+rect[3], rect[0]:rect[0]+rect[2]]
            id_, conf = recognizer.predict(roi_gray)
            
            print(max(preds))
            label = labels[id_]+" is "+class_labels[preds.argmax()] + ":" + str(round(max(preds) * 100, 2))
            label_position = (rect[0] + int((rect[1])), rect[2] + 25)
            cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)

        else:
            cv2.putText(image, "No Face found", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)

        cv2.imshow('Emotion Detection', image)
        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()


root = Tk()

root.title("Personal Face Recognition and Emotion-Recognition")
root.geometry('500x570')
tf = Frame(root, width = 480, height = 50, bd = 2, relief = "ridge")
tf.place(x = 10, y = 10)
title = Label(tf,font = ("Helvetica",30),fg = "blue",text = "       Emotion Detection      ")
title.pack() 

button1 = Button(root, command = faceRecord, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Open webcam for personal face detection", bd = 5)
button1.place(x = 20, y = 80)

button2 = Button(root, command = trainFace, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Train your face", bd = 5)
button2.place(x = 20, y = 160)

button3 = Button(root, command = personalFace, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Your Personal Face", bd = 5)
button3.place(x = 20, y = 240)

button4 = Button(root, command = detectEmotion, padx = 5, pady = 5, relief = RIDGE, width = 33, background = 'cyan',font  = ("georgia",15), text = "Emotion with your face", bd = 5)
button4.place(x = 20, y = 320)

root.mainloop()

Found 216 images belonging to 6 classes.
{0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}


c:\users\agnelselvan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:201: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


0.5577046
0.65334594
0.61936015
0.7244444
0.74880946
0.76279324
0.75207436
0.7320087
0.70742553
0.7476492
0.6707093
0.6935068
0.729263
0.7568801
0.77066207
0.8159195
0.7872267
0.8229791
0.8404441
0.84000474
0.83831286
0.8490247
0.83142906
0.860814
0.83009404
0.83428395
0.8209153
0.7911509
0.84380233
0.8277989
0.8380528
0.8231383
0.7927022
0.7826618
0.7738114
0.5247597
0.47001016
0.57354707
0.59600204
0.7316989
0.61460847
0.7408464
0.72058976
0.68837184
0.7852026
0.79681426
0.75486624
0.7463243
0.78322446
0.79323024
0.7475324
0.75307393
0.7648914
0.7631644
0.7791217
0.76134396
0.84541494
0.83680385
0.6931305
0.7594037
0.7522116
0.6878047
0.6606973
0.7373388
0.7823187
0.8254546
0.8076195
0.7644389
0.8374533
0.8338466
0.8700759
0.9006712
0.89085746
0.90314454
0.859679
0.8666711
0.8794533
0.8856491
0.89387363
0.88238144
0.8912891
0.84985805
0.85457176
0.8546544
0.8651765
0.84230936
0.8475945
0.8314406
0.8470923
0.87255865
0.8384464
0.8942609
0.8412971
0.82918686
0.8611083
0.8469281
0.87778

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\agnelselvan\appdata\local\programs\python\python37\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-cd133beb0fb4>", line 222, in detectEmotion
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

